In [260]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import os
from pathlib import Path
import re
import pickle
# ver qtos o modelo apenas de ner acertaria
from transformers import AutoTokenizer, AutoModelForTokenClassification
import nltk    
from nltk import tokenize 
import torch
from transformers import BertTokenizer,BertForTokenClassification
import numpy as np
import json   
from importlib import reload  # Python 3.4+
import random
from model import BertForChunkClassification
from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup

from torch.utils.data import DataLoader
from importlib import reload 
#from eval import predict
import eval
#import importlib
#importlib.reload(module)
from dataset import InputFeatures, load_and_cache_examples
import dataset
import functionsAval as f

In [261]:
f = reload(f)
reload(dataset)
reload(eval)

path_model=r'.\model-multilabel-io\model'

In [262]:
tags, tokens = f.predictBERTNER_multi_IO(['The HB24 mRNA was absent or present at low levels in normal B and T lymphocytes ; however , with the appropriate activation signal HB24 mRNA was induced within several hours even in the presence of cycloheximide .'], path_model)
print('-----------')
print(tags)
print(tokens)

idx2tag: {0: 'RNA', 1: 'protein-protein', 2: 'RNA-cell_type', 3: 'protein-cell_line', 4: 'DNA-RNA', 5: 'cell_line-cell_line', 6: 'RNA-RNA', 7: 'cell_type-cell_type', 8: 'protein', 9: 'protein-DNA', 10: 'DNA-cell_line', 11: 'cell_line-RNA', 12: 'protein-cell_type', 13: 'O', 14: 'cell_line-DNA', 15: 'cell_type', 16: 'DNA', 17: 'cell_type-DNA', 18: 'cell_line', 19: 'DNA-DNA', 20: 'protein-RNA', 21: 'cell_type-cell_line', 22: '<pad>'}
-----------
[['O', 'DNA-RNA', 'RNA', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'DNA-RNA', 'RNA', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]
[['the', 'hb24', 'mrna', 'was', 'absent', 'or', 'present', 'at', 'low', 'levels', 'in', 'normal', 'b', 'and', 't', 'lymphocytes', ';', 'however', ',', 'with', 'the', 'appropriate', 'activation', 'signal', 'hb24', 'mrna', 'was', 'induced', 'within', 'several', 'hours', 'even', 'in', 'the', 'presence', 'of', 'cycloheximide', '.']]


In [381]:

def save_obj(name, obj):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    with open('../obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    try:
        with open('../obj/' + name + '.pkl', 'rb') as f:
            return pickle.load(f)
    except:
        print('ERROO')
        raise
        return None
#save_obj('dicEntidadesTesteGenia', dicEntidades)

def loadSentencesTest():
    # gabarito = dicSentences_new.pkl
    #dicSentences_train = load_obj('dic_sentencesTrain.pkl')
    #dicSentences_train[32]
    # gabarito = dicSentences_new.pkl
    print('Pegando sentencas de teste gabarito: dic_sentencesTest.pkl')
    dicSentences_new_test = load_obj('dicEntidadesTesteGenia')
    return dicSentences_new_test

#dicSentences_new_test = loadSentencesTest()
#dicSentences_new_test


In [382]:
dicSentences_new_test

{0: [' There is a single methionine codon - initiated open reading frame of 1 , 458 nt in frame with a homeobox and a CAX repeat , and the open reading frame is predicted to encode a protein of 51 , 659 daltons .',
  [[['methionine codon - initiated open reading frame',
     [4, 5, 6, 7, 8, 9, 10],
     'DNA'],
    ['homeobox', [20], 'DNA'],
    ['CAX repeat', [23, 24], 'DNA'],
    ['open reading frame', [28, 29, 30], 'DNA']]]],
 1: [' When the homeodomain from HB24 was compared to known mammalian and Drosophila homeodomains it was found to be only moderately conserved , but when it was compared to a highly diverged Drosophila homeodomain , H2 . 0 , it was found to be 80 % identical .',
  [[['homeodomain', [2], 'DNA'],
    ['HB24', [4], 'DNA'],
    ['homeodomain', [32], 'DNA']],
   [['Drosophila homeodomain', [31, 32], 'DNA']]]],
 2: [' The HB24 mRNA was absent or present at low levels in normal B and T lymphocytes ; however , with the appropriate activation signal HB24 mRNA was induce

In [383]:
#BATCH=100
BATCH=3
dicSentences_new_test = loadSentencesTest()
print(len(dicSentences_new_test))
dicSentences_new_test = {k: v for k, v in dicSentences_new_test.items() if k<BATCH}
print(dicSentences_new_test[0])
#print(dicSentences_new_test[27])
print('numero de sentencas no total:', len(dicSentences_new_test))

sentences=list()
for key, value in dicSentences_new_test.items():
    if key<BATCH:
        tokens = value[0]
        #tokens = [tok[0] for tok in tokens]
        #sentences.append(' '.join(tokens).strip())
        sentences.append(tokens)

Pegando sentencas de teste gabarito: dic_sentencesTest.pkl
1854
[' There is a single methionine codon - initiated open reading frame of 1 , 458 nt in frame with a homeobox and a CAX repeat , and the open reading frame is predicted to encode a protein of 51 , 659 daltons .', [['methionine codon - initiated open reading frame', [4, 5, 6, 7, 8, 9, 10], 'DNA'], ['homeobox', [20], 'DNA'], ['CAX repeat', [23, 24], 'DNA'], ['open reading frame', [28, 29, 30], 'DNA']]]
numero de sentencas no total: 3


In [384]:
sentences[:2]

[' There is a single methionine codon - initiated open reading frame of 1 , 458 nt in frame with a homeobox and a CAX repeat , and the open reading frame is predicted to encode a protein of 51 , 659 daltons .',
 ' When the homeodomain from HB24 was compared to known mammalian and Drosophila homeodomains it was found to be only moderately conserved , but when it was compared to a highly diverged Drosophila homeodomain , H2 . 0 , it was found to be 80 % identical .']

In [385]:
dicSentences_new_test

{0: [' There is a single methionine codon - initiated open reading frame of 1 , 458 nt in frame with a homeobox and a CAX repeat , and the open reading frame is predicted to encode a protein of 51 , 659 daltons .',
  [['methionine codon - initiated open reading frame',
    [4, 5, 6, 7, 8, 9, 10],
    'DNA'],
   ['homeobox', [20], 'DNA'],
   ['CAX repeat', [23, 24], 'DNA'],
   ['open reading frame', [28, 29, 30], 'DNA']]],
 1: [' When the homeodomain from HB24 was compared to known mammalian and Drosophila homeodomains it was found to be only moderately conserved , but when it was compared to a highly diverged Drosophila homeodomain , H2 . 0 , it was found to be 80 % identical .',
  [['homeodomain', [2], 'DNA'],
   ['HB24', [4], 'DNA'],
   ['homeodomain', [32], 'DNA'],
   ['Drosophila homeodomain', [31, 32], 'DNA']]],
 2: [' The HB24 mRNA was absent or present at low levels in normal B and T lymphocytes ; however , with the appropriate activation signal HB24 mRNA was induced within seve

In [347]:
def getDicPredictions(tags, tokens):
    listaTags=['DNA', 'RNA','cell_type', 'cell_line','protein']

    dic_predictions = {}
    i=0
    for tags_sentence, tokens_sentence in zip(tags, tokens):
        listaEntidades=[]
        isEnt=0
        labelAnterior=''
        entidade=''
        label=''
        #listaTags
        for classe in listaTags:
            num_token=0
            indices_entidade = list()
            for tag, token in zip(tags_sentence, tokens_sentence):
                #print(token, tag)
                labels = tag.split('-')
                if tag!='O' and classe in labels:
                    isEnt=1
                    indices_entidade.append(num_token)
                    #print('---label-->:', label)
                    #print('---labelAnterior-->:', labelAnterior)
                    #if label == labelAnterior:
                    if labelAnterior in classe:
                        # continuacao
                        entidade = entidade+' '+token
                    elif labelAnterior=='': # primeiro token da entidade
                        entidade=token
                        labelAnterior=classe
                    else: # mudou de entidade
                        listaEntidades.append([entidade, indices_entidade, labelAnterior])
                        indices_entidade=list()
                        entidade=token
                    labelAnterior = classe
                else:
                    if isEnt==1:
                        if entidade:
                            #print('---label-->:', label)
                            listaEntidades.append([entidade, indices_entidade, classe])
                            indices_entidade=list()
                    entidade=''
                    labelAnterior=''
                    isEnt=0
                    label=''
                num_token=num_token+1

        if len(listaEntidades)>0:
            dic_predictions[i]=['FRASE', listaEntidades]
        else:
            dic_predictions[i]=['FRASE', []]
        i=i+1

    return dic_predictions
dic_predictions = getDicPredictions(tags, tokens)
dic_predictions

{0: ['FRASE', []],
 1: ['FRASE', []],
 2: ['FRASE', []],
 3: ['FRASE', []],
 4: ['FRASE', []],
 5: ['FRASE', []],
 6: ['FRASE', []],
 7: ['FRASE', []],
 8: ['FRASE', []],
 9: ['FRASE', []],
 10: ['FRASE', []],
 11: ['FRASE', []],
 12: ['FRASE', []],
 13: ['FRASE', []],
 14: ['FRASE', []],
 15: ['FRASE', []],
 16: ['FRASE', []],
 17: ['FRASE', []],
 18: ['FRASE', []],
 19: ['FRASE', []],
 20: ['FRASE', []],
 21: ['FRASE', []],
 22: ['FRASE', []],
 23: ['FRASE', []],
 24: ['FRASE', []],
 25: ['FRASE', []],
 26: ['FRASE', []],
 27: ['FRASE', []],
 28: ['FRASE', []],
 29: ['FRASE', []],
 30: ['FRASE', []],
 31: ['FRASE', []],
 32: ['FRASE', []],
 33: ['FRASE', []],
 34: ['FRASE', []],
 35: ['FRASE', []],
 36: ['FRASE', []],
 37: ['FRASE', []],
 38: ['FRASE', []],
 39: ['FRASE', []],
 40: ['FRASE', []],
 41: ['FRASE', []],
 42: ['FRASE', []],
 43: ['FRASE', []],
 44: ['FRASE', []],
 45: ['FRASE', []],
 46: ['FRASE', []],
 47: ['FRASE', []],
 48: ['FRASE', []],
 49: ['FRASE', []],
 50: ['FRA

In [348]:
tags, tokens = f.predictBERTNER_multi_IO(sentences, path_model)
dic_predictions = getDicPredictions(tags, tokens)
print(dic_predictions[0])
print('len(dic_predictions):', len(dic_predictions))

idx2tag: {0: 'RNA', 1: 'protein-protein', 2: 'RNA-cell_type', 3: 'protein-cell_line', 4: 'DNA-RNA', 5: 'cell_line-cell_line', 6: 'RNA-RNA', 7: 'cell_type-cell_type', 8: 'protein', 9: 'protein-DNA', 10: 'DNA-cell_line', 11: 'cell_line-RNA', 12: 'protein-cell_type', 13: 'O', 14: 'cell_line-DNA', 15: 'cell_type', 16: 'DNA', 17: 'cell_type-DNA', 18: 'cell_line', 19: 'DNA-DNA', 20: 'protein-RNA', 21: 'cell_type-cell_line', 22: '<pad>'}
['FRASE', [[' methionine codon - initiated open reading frame', [4, 5, 6, 7, 8, 9, 10], 'DNA'], [' homeobox', [20], 'DNA'], [' cax repeat', [23, 24], 'DNA'], [' open reading frame', [28, 29, 30], 'DNA']]]
len(dic_predictions): 3


In [370]:
dic_predictions[0]

['FRASE',
 [[' methionine codon - initiated open reading frame',
   [4, 5, 6, 7, 8, 9, 10],
   'DNA'],
  [' homeobox', [20], 'DNA'],
  [' cax repeat', [23, 24], 'DNA'],
  [' open reading frame', [28, 29, 30], 'DNA']]]

In [371]:
dicSentences_new_test[0]

[' There is a single methionine codon - initiated open reading frame of 1 , 458 nt in frame with a homeobox and a CAX repeat , and the open reading frame is predicted to encode a protein of 51 , 659 daltons .',
 [[['methionine codon - initiated open reading frame',
    [4, 5, 6, 7, 8, 9, 10],
    'DNA'],
   ['homeobox', [20], 'DNA'],
   ['CAX repeat', [23, 24], 'DNA'],
   ['open reading frame', [28, 29, 30], 'DNA']]]]

In [372]:
dicSentences_new_test[0][1][0]

[['methionine codon - initiated open reading frame',
  [4, 5, 6, 7, 8, 9, 10],
  'DNA'],
 ['homeobox', [20], 'DNA'],
 ['CAX repeat', [23, 24], 'DNA'],
 ['open reading frame', [28, 29, 30], 'DNA']]

In [386]:
def AvalFinal(dicSentences_new_test, dic_predictions_all, BATCH):

    region_true_list, region_pred_list = list(), list() # labels
    region_true_count, region_pred_count = 0, 0 # contagem
    numErro1=0
    numErro2=0

    for i in range(0, BATCH, 1):
        if i<len(dicSentences_new_test):
        #if 1==1:           
            #print('\n---Label vs predicao--------')
            #print('frase:', dicSentences_new_test[i][0])
            #print('i:', i)
            #print(dicSentences_new_test[i][1])
            #print(dic_predictions_all[i][1])
            entidades_gabarito = dicSentences_new_test[i][1]
            try:
                #print(dic_predictions_all[i])
                entidades_preditas = dic_predictions_all[i][1]
            except:
                print('erro, caiu no except na AvalFinal:', str(i))
                raise
            #print('entidades_preditas:', entidades_preditas)
            for entidade_gabarito in entidades_gabarito:
                #entidade_gabarito=entidade_gabarito[0]
                indices_gabarito = entidade_gabarito[1]
                tag_gabarito = entidade_gabarito[2]
                region_true_count=region_true_count+1
                region_true_list.append(tag_gabarito)
                # ver se previu essa entidade
                previu=0
                for entidade_predita in entidades_preditas:
                    indices_predita = entidade_predita[1]
                    tag_predita = entidade_predita[2]
                    if indices_predita == indices_gabarito:
                        region_pred_list.append(tag_predita)
                        previu=1
                        print('achou')
                        if tag_predita !='O':
                            region_pred_count=region_pred_count+1
                        break
                if previu==0:
                    numErro1=numErro1+1
                    region_pred_list.append('O')

            # agora o contrario, ver o q previu mas nao era

            for entidade_predita in entidades_preditas:
                indices_predita = entidade_predita[1]
                tag_predita = entidade_predita[2]
                # ver se a entidade prevista existe ou é FP
                existe=0
                for entidade_gabarito in entidades_gabarito:
                    #entidade_gabarito=entidade_gabarito[0]
                    indices_gabarito = entidade_gabarito[1]
                    if indices_predita == indices_gabarito:
                        existe=1
                        break
                if existe==0:
                    numErro2 = numErro2+1
                    region_true_list.append('O')
                    region_pred_list.append(tag_predita)

    #print('===resultados====')
    #print('region_true_list:', region_true_list)
    #print('region_pred_list:', region_pred_list)

    print('numErro1:', numErro1)
    print('numErro2:', numErro2)
    #print(classification_report(region_true_list, region_pred_list, labels=['O', 'PROTEIN', 'DNA', 'RNA', 'CELL_TYPE', 'CELL_LINE'], target_names=['O', 'PROTEIN', 'DNA', 'RNA', 'CELL_TYPE', 'CELL_LINE'], digits=6))
    print(classification_report(region_true_list, region_pred_list, digits=6))

    return region_true_list, region_pred_list

region_true_list, region_pred_list = AvalFinal(dicSentences_new_test, dic_predictions, BATCH)


achou
achou
achou
achou
achou
achou
achou
achou
achou
numErro1: 2
numErro2: 3
              precision    recall  f1-score   support

         DNA   0.700000  0.875000  0.777778         8
           O   0.000000  0.000000  0.000000         3
         RNA   1.000000  1.000000  1.000000         2
   cell_type   0.000000  0.000000  0.000000         1

    accuracy                       0.642857        14
   macro avg   0.425000  0.468750  0.444444        14
weighted avg   0.542857  0.642857  0.587302        14



In [374]:
#dic_predictions[5]

In [375]:
region_true_list[0:30]

['DNA',
 'DNA',
 'DNA',
 'DNA',
 'DNA',
 'DNA',
 'DNA',
 'O',
 'O',
 'RNA',
 'RNA',
 'O',
 'O']

In [376]:
region_pred_list[0:30]

['DNA',
 'DNA',
 'DNA',
 'DNA',
 'DNA',
 'DNA',
 'O',
 'DNA',
 'DNA',
 'RNA',
 'RNA',
 'DNA',
 'DNA']

In [380]:
dicSentences_new_test

{0: [' There is a single methionine codon - initiated open reading frame of 1 , 458 nt in frame with a homeobox and a CAX repeat , and the open reading frame is predicted to encode a protein of 51 , 659 daltons .',
  [[['methionine codon - initiated open reading frame',
     [4, 5, 6, 7, 8, 9, 10],
     'DNA'],
    ['homeobox', [20], 'DNA'],
    ['CAX repeat', [23, 24], 'DNA'],
    ['open reading frame', [28, 29, 30], 'DNA']]]],
 1: [' When the homeodomain from HB24 was compared to known mammalian and Drosophila homeodomains it was found to be only moderately conserved , but when it was compared to a highly diverged Drosophila homeodomain , H2 . 0 , it was found to be 80 % identical .',
  [[['homeodomain', [2], 'DNA'],
    ['HB24', [4], 'DNA'],
    ['homeodomain', [32], 'DNA']],
   [['Drosophila homeodomain', [31, 32], 'DNA']]]],
 2: [' The HB24 mRNA was absent or present at low levels in normal B and T lymphocytes ; however , with the appropriate activation signal HB24 mRNA was induce

In [358]:
dic_predictions

{0: ['FRASE',
  [[' methionine codon - initiated open reading frame',
    [4, 5, 6, 7, 8, 9, 10],
    'DNA'],
   [' homeobox', [20], 'DNA'],
   [' cax repeat', [23, 24], 'DNA'],
   [' open reading frame', [28, 29, 30], 'DNA']]],
 1: ['FRASE',
  [[' homeodomain', [2], 'DNA'],
   [' hb24', [4], 'DNA'],
   [' drosophila homeodomain', [31, 32], 'DNA'],
   [' h2 . 0', [34, 35, 36], 'DNA']]],
 2: ['FRASE',
  [[' hb24', [1], 'DNA'],
   [' hb24', [24], 'DNA'],
   [' hb24 mrna', [1, 2], 'RNA'],
   [' hb24 mrna', [24, 25], 'RNA']]]}

In [ ]:
region_true_list, region_pred_list = AvalFinal(dicSentences_new_test, dic_predictions, BATCH)
print(confusion_matrix(region_true_list, region_pred_list))